# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 27 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [ ]:
!pip install torchsummaryX wandb --quiet

In [ ]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)
from torch.optim.lr_scheduler import StepLR  # Add this import statement
from tqdm import tqdm

Device:  cuda


In [ ]:
# ### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"arushijain99","key":"d8d74b4e9db19caa2853582b50318479"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# commands to download data from kaggle
!kaggle competitions download -c 11785-hw1p2-s24

!unzip -qo /content/11785-hw1p2-s24.zip -d '/content'

11785-hw1p2-s24.zip: Skipping, found more recently modified local copy (use --force to force download)


# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [ ]:

# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes


        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = os.path.join(root, partition, "mfcc")
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root

        self.transcript_dir = os.path.join(root,partition,"transcript")

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    = sorted(os.listdir(self.transcript_dir))
        # if len(mfcc_names) > 20000:
        #   mfcc_names = mfcc_names[:19000]
        #   transcript_names = transcript_names[:19000]

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        =  np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mean = np.mean(mfcc, axis=0)
            std = np.std(mfcc, axis=0)
            mfcc = (mfcc - mean) / (std)
        #   Load the corresponding transcript

            transcript=np.load(os.path.join(self.transcript_dir,transcript_names[i]))[1:-1]

            #with open(transcript_path, 'r') as file:
              #transcript = file.read().strip().split()
            #transcript  = NotImplemented # Remove [SOS] and [EOS] from the transcript
            #transcript = transcript[1:-1]
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2), (T2+2) before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 27 (Where T = T1 + T2 + ...)
        self.mfccs          = np.concatenate(self.mfccs, axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts    = np.concatenate(self.transcripts)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is.
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((context, context), (0, 0)), mode='constant', constant_values=0) # TODO

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers
        unique_phonemes = {phoneme:i for i,phoneme in enumerate(PHONEMES)}

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        self.transcripts = np.array([unique_phonemes[phoneme] for phoneme in self.transcripts])

        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind: ind + 2 * self.context + 1, :]
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[ind])

        return frames, phonemes

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, phonemes=PHONEMES, context=0, partition="test-clean"):  # Fix: Double underscores
        self.context = context
        self.mfcc_dir = os.path.join(root, 'mfcc')
        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        self.mfccs = []
        for i in range(len(mfcc_names)):
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
            mean = np.mean(mfcc, axis= 0)
            std = np.std(mfcc, axis=0)
            mfcc = (mfcc - mean) / std
            self.mfccs.append(mfcc)
        self.mfccs = np.concatenate(self.mfccs)
        self.length = len(self.mfccs)
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), 'constant', constant_values=(0, 0))

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        start = ind
        end = ind + (2 * self.context) + 1
        frames = self.mfccs[start:end, :]
        frames = np.ndarray.flatten(frames)
        frames = torch.FloatTensor(frames)
        return frames


# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [ ]:
config = {
    'epochs'        : 55,
    #'batch_size'    : 1024,
    'batch_size'    : 2048,
    'context'       : 30,

    'init_lr'       : 1e-3,
    'architecture'  : 'Pyramid',
    #'dropout'       : 0.15,
    # 'betas'         :(0.9, 0.999),
    # 'eps'           : 1e-8,
    # 'weight_decay'  : 0.1,
    # 'amsgrad'       : False
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [ ]:

#TODO: Create a dataset object using the AudioDataset class for the training  data /content/11-785-s24-hw1p2/dev-clean
train_data=AudioDataset(root="/content/11-785-s24-hw1p2", context=config['context'], partition = "train-clean-100")

# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset(root= "/content/11-785-s24-hw1p2", context=config['context'], partition="dev-clean")

# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset(root="/content/11-785-s24-hw1p2/test-clean", context=config['context'], partition="test-clean")

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  2048
Context        :  30
Input size     :  1647
Output symbols :  42
Train dataset samples = 36091157, batches = 17623
Validation dataset samples = 1928204, batches = 942
Test dataset samples = 1934138, batches = 945


In [ ]:
train_data.length

36091157

In [ ]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 1647]) torch.Size([2048])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [ ]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff

class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()

        self.model = torch.nn.Sequential(

            # torch.nn.Linear(input_size, 1647),
            # torch.nn.BatchNorm1d(1647),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p=dropout),
            # torch.nn.Linear(1647, 1647),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p=dropout),
            # torch.nn.Linear(1647, 1647),
            # torch.nn.BatchNorm1d(1647),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p=dropout),
            # torch.nn.Linear(1647, 1500),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p=dropout),
            # torch.nn.Linear(1500, 1500),
            # torch.nn.BatchNorm1d(1500),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p=dropout),
            # torch.nn.Linear(1500, 1500),
            # torch.nn.BatchNorm1d(1500),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p=dropout),
            # torch.nn.Linear(1500, 1500),
            # torch.nn.BatchNorm1d(1500),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p=dropout),
            # torch.nn.Linear(1500, 500),
            # torch.nn.BatchNorm1d(500),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p=dropout),
            # torch.nn.Linear(500, 300),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p=dropout),
            # torch.nn.Linear(300, 200),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p=dropout),
            # torch.nn.Linear(200, 100),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p=dropout),
            # torch.nn.Linear(100,42),
            # torch.nn.GELU(),
            # torch.nn.Dropout(p='dropout'),
            # torch.nn.Linear(42, output_size)

        #     torch.nn.Linear(input_size, 1647),
        #     torch.nn.BatchNorm1d(1647),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1647, 1647),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1647, 1647),
        #     torch.nn.BatchNorm1d(1647),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1647, 1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500, 1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500, 1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500, 1500),
        #     torch.nn.BatchNorm1d(1500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(1500, 500),
        #     torch.nn.BatchNorm1d(500),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(500, 300),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(300, 200),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(200, 100),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(100, 42),
        #     torch.nn.GELU(),
        #     torch.nn.Dropout(p = config['dropout']),
        #     torch.nn.Linear(42, output_size)

            torch.nn.Linear(input_size, 1647),
            torch.nn.BatchNorm1d(1647),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.2),
            torch.nn.Linear(1647, 1647),
            torch.nn.BatchNorm1d(1647),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.2),
            torch.nn.Linear(1647, 1647),
            torch.nn.BatchNorm1d(1647),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.2),
            torch.nn.Linear(1647, 1647),
            torch.nn.BatchNorm1d(1647),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.2),
            torch.nn.Linear(1647, 1500),
            torch.nn.BatchNorm1d(1500),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.15),
            torch.nn.Linear(1500, 1500),
            torch.nn.BatchNorm1d(1500),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.15),
            torch.nn.Linear(1500, 1000),
            torch.nn.BatchNorm1d(1000),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.1),
            torch.nn.Linear(1000, 1000),
            torch.nn.BatchNorm1d(1000),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.1),
            torch.nn.Linear(1000, 1000),
            torch.nn.BatchNorm1d(1000),
            torch.nn.GELU(),
            torch.nn.Linear(1000, 1000),
            torch.nn.BatchNorm1d(1000),
            torch.nn.Dropout(p = 0.1),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.1),
            torch.nn.Linear(1000, 1000),
            torch.nn.BatchNorm1d(1000),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.1),
            torch.nn.Linear(1000, 1000),
            torch.nn.BatchNorm1d(1000),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.1),
            torch.nn.Linear(1000, 100),
            torch.nn.BatchNorm1d(100),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.05),
            torch.nn.Linear(100, 42),
            torch.nn.BatchNorm1d(42),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.05),
            torch.nn.Linear(42, output_size)

          # torch.nn.Linear(input_size, 1140),
          #   torch.nn.BatchNorm1d(1140),
          #   torch.nn.ReLU(),
          #   torch.nn.Dropout(p = config['dropout']),
          #   torch.nn.Linear(1140, 1140),
          #   torch.nn.BatchNorm1d(1140),
          #   torch.nn.ReLU(),
          #   torch.nn.Dropout(p = config['dropout']),
          #   torch.nn.Linear(1140, 1140),
          #   torch.nn.BatchNorm1d(1140),
          #   torch.nn.ReLU(),
          #   torch.nn.Dropout(p = config['dropout']),
          #   torch.nn.Linear(1140, 1140),
          #   torch.nn.BatchNorm1d(1140),
          #   torch.nn.ReLU(),
          #   torch.nn.Dropout(p = config['dropout']),
          #   torch.nn.Linear(1140, 1140),
          #   torch.nn.BatchNorm1d(1140),
          #   torch.nn.ReLU(),
          #   torch.nn.Dropout(p = config['dropout']),
          #   torch.nn.Linear(1140, 1140),
          #   torch.nn.BatchNorm1d(1140),
          #   torch.nn.ReLU(),
          #   torch.nn.Dropout(p = config['dropout']),
          #   torch.nn.Linear(1140, 1140),
          #   torch.nn.BatchNorm1d(1140),
          #   torch.nn.ReLU(),
          #   torch.nn.Dropout(p = config['dropout']),
          #   torch.nn.Linear(1140, 1140),
          #   torch.nn.ReLU(),
          #   torch.nn.Linear(1140, 50),
          #   torch.nn.BatchNorm1d(50),
          #   torch.nn.ReLU(),
          #   torch.nn.Linear(50, output_size),









         )

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [ ]:
INPUT_SIZE  = (2*config['context'] + 1) * 27 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 24 million parameters for HW1 (including ensembles)

                         Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                                    
0_model.Linear_0         [1647, 1647]  [2048, 1647]  2.714256M  2.712609M
1_model.BatchNorm1d_1          [1647]  [2048, 1647]     3.294k     1.647k
2_model.GELU_2                      -  [2048, 1647]          -          -
3_model.Dropout_3                   -  [2048, 1647]          -          -
4_model.Linear_4         [1647, 1647]  [2048, 1647]  2.714256M  2.712609M
5_model.BatchNorm1d_5          [1647]  [2048, 1647]     3.294k     1.647k
6_model.GELU_6                      -  [2048, 1647]          -          -
7_model.Dropout_7                   -  [2048, 1647]          -          -
8_model.Linear_8         [1647, 1647]  [2048, 1647]  2.714256M  2.712609M
9_model.BatchNorm1d_9          [1647]  [2048, 1647]     3.294k     1.647k
10_model.GELU_10                    -  [2048, 1647]          -          -
11_model.Dropout_11                 - 

/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[1647, 1647]","[2048, 1647]",2714256.0,2712609.0
1_model.BatchNorm1d_1,[1647],"[2048, 1647]",3294.0,1647.0
2_model.GELU_2,-,"[2048, 1647]",NaN,NaN
3_model.Dropout_3,-,"[2048, 1647]",NaN,NaN
4_model.Linear_4,"[1647, 1647]","[2048, 1647]",2714256.0,2712609.0
5_model.BatchNorm1d_5,[1647],"[2048, 1647]",3294.0,1647.0
6_model.GELU_6,-,"[2048, 1647]",NaN,NaN
7_model.Dropout_7,-,"[2048, 1647]",NaN,NaN
8_model.Linear_8,"[1647, 1647]","[2048, 1647]",2714256.0,2712609.0


In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.optim.lr_scheduler as lr_scheduler


criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

#optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr'])
#optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01, amsgrad=False)
optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01, amsgrad=False)

#optimizer=torch.optim.SGD(model.parameters(), lr=config['init_lr'], momentum=0.5)
#scheduler = StepLR(optimizer, step_size=3, gamma = 0.3)
#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True, min_lr=1e-6)
scheduler_cos = CosineAnnealingLR(optimizer, T_max=config['epochs'])
# milestones = [5,10,15, 20,25, 28, 30,35]  # You can adjust the milestones as needed
# scheduler_multistep = lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.5)

scaler = torch.cuda.amp.GradScaler()

#Defining Optimizer
# Recommended : Define Scheduler for Learning Rate,
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html


# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

36

In [ ]:
from torch.cuda.amp import autocast, GradScaler

def train(model, dataloader, optimizer, criterion, scheduler_cos, scaler):
#def train(model, dataloader, optimizer, criterion, scheduler_cos):


    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        optimizer.zero_grad()

        with autocast():
            frames      = frames.to(device)
            phonemes    = phonemes.to(device)

            logits  = model(frames)
            loss    = criterion(logits, phonemes)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        del frames, phonemes, logits
        torch.cuda.empty_cache()

    #cosine_scheduler.step()
    # #reduce_lr_scheduler.step(tloss)
    scheduler_cos.step()
    # scheduler_multistep.step()

    batch_bar.close()
    tloss   /= len(dataloader)
    tacc    /= len(dataloader)

    return tloss, tacc

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    #scheduler.step()
    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="66a818a0cbd3bb3492e8f908b65fec7b4ed03afc") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name    = "AJ_T13", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

wandb: Currently logged in as: arushij (idl1). Use `wandb login --relogin` to force relogin


In [ ]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20240210_011900-vf1h3viz/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
scaler = GradScaler()
import os

# Directory to save the checkpoints
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Path to the checkpoint of the 40th epoch
checkpoint_path = os.path.join(checkpoint_dir, "epoch_28.pt")

# Load the checkpoint
checkpoint = torch.load(checkpoint_path)

# Load the model state dictionary and optimizer state dictionary
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Get the epoch number from the checkpoint
epoch = checkpoint['epoch']

for epoch in range(config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion, scheduler_cos, scaler)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    # Log metrics at each epoch in your run
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    # Save checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f"epoch_{epoch+1}.pt")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        'train_acc': train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc,
        'lr': curr_lr
    }, checkpoint_path)

# Finish your wandb run
run.finish()


Epoch 1/55


	Train Acc 87.1022%	Train Loss 0.3693	 Learning Rate 0.0004857
	Val Acc 86.0243%	Val Loss 0.4284

Epoch 2/55


	Train Acc 87.1417%	Train Loss 0.3680	 Learning Rate 0.0004853
	Val Acc 86.0304%	Val Loss 0.4282

Epoch 3/55


	Train Acc 87.1475%	Train Loss 0.3676	 Learning Rate 0.0004841
	Val Acc 86.0194%	Val Loss 0.4278

Epoch 4/55


	Train Acc 87.1590%	Train Loss 0.3671	 Learning Rate 0.0004822
	Val Acc 86.0803%	Val Loss 0.4260

Epoch 5/55


...

	Train Acc 87.2430%	Train Loss 0.3643	 Learning Rate 0.0004666
	Val Acc 86.1269%	Val Loss 0.4255

Epoch 9/55


	Train Acc 87.2650%	Train Loss 0.3634	 Learning Rate 0.0004608
	Val Acc 86.1529%	Val Loss 0.4240

Epoch 10/55


	Train Acc 87.3027%	Train Loss 0.3621	 Learning Rate 0.0004543
	Val Acc 86.2129%	Val Loss 0.4245

Epoch 11/55


Train:  19%|█▉        | 3330/17623 [02:43<13:00, 18.32it/s, acc=87.4813%, loss=0.3560]

KeyboardInterrupt: 

In [ ]:
!ls /content/checkpoints/

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
# @title Default title text
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval()# TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.no_grad(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phoneme_indices = torch.argmax(logits, dim = 1)
            predicted_phonemes = [PHONEMES[i] for i in predicted_phoneme_indices]

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            # TODO
            test_predictions.extend(predicted_phonemes)

            if i<5:
              print([PHONEMES[i] for i in predicted_phoneme_indices])


    return test_predictions

In [ ]:
# Create your wandb run
run = wandb.init(
    name    = "Cyl-Pyr-BatchNorm-SGD", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

lr,███▇▇▆▅▄▂▁
train_acc,▁▂▃▃▃▄▅▆▇█
train_loss,█▇▆▆▆▅▄▃▂▁
val_acc,▁▁▁▃▄▂▅▅▆█
valid_loss,██▇▄▅▅▃▃▁▂
lr,0.00045
train_acc,87.30272
train_loss,0.36209
val_acc,86.21288
valid_loss,0.42445


In [ ]:
predictions = test(model, test_loader)


  0%|          | 3/945 [00:00<02:27,  6.38it/s]

['[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', '[SIL]', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'IY', 'IY', 'IY', 'IY', 'IY', 'IY', 'IY', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'OW', 'OW', 'OW', 'OW', 'OW', 'OW', 'OW', 'OW', 'OW', 'OW', 'OW', 'OW', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'DH', 'DH', 'DH', 'DH', 'DH', 'EH', 'EH', 'EH', 'EH', 'R', 'R', 'R', 'R', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'UH', 'UH', 'UH', 'UH', 'UH', 'UH', 'D', 'D', 'D', 'D', 'D', 'B', 'B', 'B', 'B', 'B', 'IY', 'IY', 'IY', 'IY', 'IY', 'IY', 'IY', 'IY', 'IY', 'S', 'S', 'S', 'S', '


  1%|          | 5/945 [00:00<01:56,  8.10it/s]

['AH', 'AH', 'AH', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'T', 'T', 'T', 'T', 'T', 'T', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'AH', 'AH', 'AH', 'AH', 'AH', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'EH', 'EH', 'EH', 'EH', 'EH', 'EH', 'EH', 'EH', 'EH', 'EH', 'EH', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'ER', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'UW', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'B', 'B', 'B', 'B', 'B', 'B', 'L', 'L', 'L', 'L', 'L', 'L', 'EH', 'EH', 'EH', 'EH', 'EH', 'EH', 'EH', 'EH', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'IH', 'IH', 'IH', 'IH', 'NG', 'NG', 'NG', 'NG', 'NG', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'HH', 'IY', 'IY', 'IY', 'IY', 'IY', 'IY', 'IY', 'IY', 'IY', 'F', 'F',


100%|██████████| 945/945 [00:46<00:00, 20.17it/s]


In [ ]:
print(predictions[:100])

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11785-hw1p2-s24 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle

100% 19.3M/19.3M [00:03<00:00, 5.75MB/s]
Successfully submitted to 11785-HW1P2-S24